# [3] 분석 대상 선정
 1) 배송 지연율이 10% 이상인 지역 확인  
 2) 가중평균을 통한 우선 순위 지역 선정

## 1) 배송 지연율 10% 이상인 지역 확인

In [ ]:
# 배송이 완료된(order_status=delivered) 데이터 추출 ; delivered
delivered = raw[raw['order_status'] == 'delivered'].copy()
delivered.head(2)
delivered.shape

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,product_height_cm,product_width_cm,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,is_delayed,delay_days
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,8.0,13.0,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,0,0.0
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,8.0,13.0,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,0,0.0


(115723, 38)

In [ ]:
# state별 배송 지연율 계산 : order_id 기준 중복 제거 안함
state_delay = delivered.groupby('customer_state')['is_delayed'].agg(['mean', 'count'])
state_delay['delay_percent'] = state_delay['mean'] * 100


# 지연율 10% 이상인 state 추출
high_delay_states = state_delay[state_delay['delay_percent'] >= 10].sort_values(by = 'delay_percent', ascending = False).reset_index()
high_delay_states

,customer_state,mean,count,delay_percent
0,AL,0.244944,445,24.494382
1,MA,0.201701,823,20.170109
2,PI,0.158845,554,15.884477
3,SE,0.158031,386,15.803109
4,CE,0.150266,1504,15.026596
5,BA,0.137468,3950,13.746835
6,RJ,0.129386,14963,12.938582
7,PA,0.123063,1097,12.306290
8,ES,0.121225,2318,12.122519
9,RN,0.115453,563,11.545293


## 2) 가중평균 우선순위 지역 선정
> 배송지연율이 10% 이상인 state 내에서  
> 최종 소비자수(50%)와 매출액(50%) 가중평균으로 우선적 state (RJ, BA) 선정
- 장기적인 손익(소비자수)과 단기적인 손익(매출액)을 동시에 반영
-  소비자 수와 매출액은 각기 다른 시점에서 중요도를 가지므로 둘 중 하나에 편중되면 불균형이 발생할 수 있다고 생각
- 5:5 균등 가중치를 부여함으로써 단기 손익과 장기 성장가능성을 모두 고려한 균형 있는 지역 선정을 하고자 함

In [ ]:
# state별 고객수와 매출액 집계
customer_count = delivered.groupby('customer_state')['customer_id'].nunique().reset_index(name='customer_count')
total_sales = delivered.groupby('customer_state')['payment_value'].sum().reset_index(name='total_sales')
state_summary = customer_count.merge(total_sales, on='customer_state')
state_summary

,customer_state,customer_count,total_sales
0,AC,80,24890.49
1,AL,397,108727.76
2,AM,145,34586.18
3,AP,67,21521.71
4,BA,3256,775836.28
5,CE,1279,335018.17
6,DF,2080,424872.44
7,ES,1995,399308.36
8,GO,1957,497367.84
9,MA,717,195528.31


In [ ]:
# 지연율 10% 이상 state만 필터링
filtered_state_summary = state_summary[state_summary['customer_state'].isin(high_delay_states['customer_state'])]

In [ ]:
# 고객 수와 매출액 표준화
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

filtered_state_summary['customer_scaled'] = min_max_scaling(filtered_state_summary['customer_count'])
filtered_state_summary['sales_scaled'] = min_max_scaling(filtered_state_summary['total_sales'])

In [ ]:
# 가중평균 계산
filtered_state_summary['weighted_avg'] = (0.5 * filtered_state_summary['customer_scaled'] + 0.5 * filtered_state_summary['sales_scaled'])
filtered_state_summary = filtered_state_summary.sort_values(by = 'weighted_avg', ascending = False).reset_index()

filtered_state_summary

,index,customer_state,mean,count,delay_percent,customer_count,total_sales,customer_scaled,sales_scaled,weighted_avg
0,6,RJ,0.129386,14963,12.938582,12350,2699623.08,1.000000,1.000000,1.000000
1,5,BA,0.137468,3950,13.746835,3256,775836.28,0.261191,0.284391,0.272791
2,8,ES,0.121225,2318,12.122519,1995,399308.36,0.158746,0.144331,0.151538
3,14,PE,0.102174,1840,10.217391,1593,362434.77,0.126087,0.130615,0.128351
4,4,CE,0.150266,1504,15.026596,1279,335018.17,0.100577,0.120416,0.110496
5,7,PA,0.123063,1097,12.306290,946,255609.57,0.073523,0.090878,0.082201
6,1,MA,0.201701,823,20.170109,717,195528.31,0.054919,0.068529,0.061724
7,13,MS,0.106132,848,10.613208,701,164556.97,0.053619,0.057008,0.055314
8,10,PB,0.115385,624,11.538462,517,177485.32,0.038671,0.061817,0.050244
9,2,PI,0.158845,554,15.884477,476,133729.03,0.035340,0.045541,0.040440
